In [1]:
import pandas as pd
import numpy as np
import glob
import tensorflow as tf   
from sklearn.preprocessing import scale
from tensorflow.keras.models import Model
from tensorflow.keras import layers, losses

In [2]:
tf.__version__

'2.0.1'

In [3]:
tf.test.is_gpu_available()

True

In [5]:
datas = glob.glob('transfer_gps/*/*/*.txt')
train_data = []
train_label = []
for data in datas:
    f = np.loadtxt(data,delimiter=",").copy()
    f.resize((10,4))
    train_data.append(f)
    if data.split('/')[1] == "indoor":
        train_label.append("indoor"+data.split('/')[-2])
    else:
        train_label.append("outdoor"+data.split('/')[-2])
        

train_data = np.array(train_data)
train_label = pd.get_dummies(train_label).values.argmax(1)
train_label = train_label.reshape(len(train_label),1)

train_val_split = np.random.rand(len(train_data)) < 0.70
train_x = train_data[train_val_split]
train_y = train_label[train_val_split]
val_x = train_data[~train_val_split]
val_y = train_label[~train_val_split]

In [26]:
train_x.shape,train_y.shape,len(val_x)#.shape

((3179, 10, 4), (3179, 1), 1344)

In [30]:
BUFFER_SIZE = 4523
BATCH_SIZE = 25
train_dataset = tf.data.Dataset.from_tensor_slices((train_x,train_x))
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = tf.data.Dataset.from_tensor_slices((val_x,val_x)).batch(len(val_x))

In [21]:
x = tf.data.Dataset.from_tensor_slices((train_x,train_x))
x

<TensorSliceDataset shapes: ((10, 4), (10, 4)), types: (tf.float64, tf.float64)>

In [23]:
x = x.shuffle(BUFFER_SIZE)
x

<ShuffleDataset shapes: ((10, 4), (10, 4)), types: (tf.float64, tf.float64)>

In [24]:
x = x.batch(BATCH_SIZE)
x

<BatchDataset shapes: ((None, 10, 4), (None, 10, 4)), types: (tf.float64, tf.float64)>

In [25]:
x = x.repeat()
x

<RepeatDataset shapes: ((None, 10, 4), (None, 10, 4)), types: (tf.float64, tf.float64)>

In [14]:
 tf.data.Dataset.from_tensor_slices((val_x,val_x))

<TensorSliceDataset shapes: ((10, 4), (10, 4)), types: (tf.float64, tf.float64)>

In [8]:
input_o = tf.keras.layers.Input(shape=(4), name='input_layer')
models = layers.Dense(4, activation='relu')(input_o )
models = layers.Dense(4, activation='relu')(models)
models = layers.Dense(4, activation='relu')(models)
models = layers.Dense(4, activation='relu')(models)
model = tf.keras.Model(inputs=input_o, outputs=models)
model.summary()
input = tf.keras.layers.Input(shape=(10,4), name='input_layer1')
input1 = layers.Lambda(lambda x: x[:,0,:], output_shape=(1))(input)
input2 = layers.Lambda(lambda x: x[:,1,:], output_shape=(1))(input)
input3 = layers.Lambda(lambda x: x[:,2,:], output_shape=(1))(input)
input4 = layers.Lambda(lambda x: x[:,3,:], output_shape=(1))(input)
input5 = layers.Lambda(lambda x: x[:,4,:], output_shape=(1))(input)
input6 = layers.Lambda(lambda x: x[:,5,:], output_shape=(1))(input)
input7 = layers.Lambda(lambda x: x[:,6,:], output_shape=(1))(input)
input8 = layers.Lambda(lambda x: x[:,7,:], output_shape=(1))(input)
input9 = layers.Lambda(lambda x: x[:,8,:], output_shape=(1))(input)
input10= layers.Lambda(lambda x: x[:,9,:], output_shape=(1))(input)

model_1 = model(input1)
model_2 = model(input2)
model_3 = model(input3)
model_4 = model(input4)
model_5 = model(input5)
model_6 = model(input6)
model_7 = model(input7)
model_8 = model(input8)
model_9 = model(input9)
model_10= model(input10)
merge_layer = tf.keras.layers.concatenate(inputs=[model_1, model_2,model_3,model_4,model_5,model_6,model_7,model_8,model_9,model_10])
model_encoder = tf.keras.layers.Dense(32, activation='relu')(merge_layer)
model_encoder = tf.keras.layers.Dense(32, activation='relu')(model_encoder)
model_encoder = tf.keras.layers.Dense(16, activation='relu')(model_encoder)
model_down = tf.keras.Model(inputs=[input], outputs=model_encoder,name = "encoder")#input1, input2,input3,input4,input5,input6,input7,input8,input9,input10
model_down.summary()

input_encoder = tf.keras.layers.Input(shape=(10,4), name='input_layer2')
input_decoder = model_down(input_encoder)
model_decoder = layers.Dense(16, activation='relu')(input_decoder)
model_decoder = layers.Dense(32, activation='relu')(model_decoder)
model_decoder = layers.Dense(32, activation='relu')(model_decoder)
model_decoder = layers.Dense(40, activation='sigmoid')(model_decoder)
model_decoder = layers.Reshape((10,4))(model_decoder)
model_encoder_decoder = tf.keras.Model(inputs=[input_encoder],outputs=model_decoder,name = 'encoder_decoder')
model_encoder_decoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     [(None, 4)]               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 20        
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 20        
Total params: 80
Trainable params: 80
Non-trainable params: 0
_________________________________________________________________
Model: "encoder"
__________________________________________________________________________________________________
Layer (type)   

In [12]:
encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(256, activation='relu'),
      layers.Dense(128, activation='relu'),
      layers.Dense(32, activation='relu'),
    ])
encoder.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or calling `fit()` with some data, or specify an `input_shape` argument in the first layer(s) for automatic build.

In [9]:
model_encoder_decoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

In [ ]:
model_encoder_decoder.fit(train_dataset,
                epochs=1000,
                validation_data=valid_dataset)

Epoch 1/1000
128/128 [==============================] - 3s 25ms/step - loss: 1009043453120772506124288.0000 - val_loss: 0.0000e+00
Epoch 2/1000
128/128 [==============================] - 3s 20ms/step - loss: 1016978681743773662183424.0000 - val_loss: 1134468995925614390149120.0000
Epoch 3/1000
128/128 [==============================] - 3s 20ms/step - loss: 1016978680029060716822528.0000 - val_loss: 1134468995925614390149120.0000
Epoch 4/1000
128/128 [==============================] - 3s 21ms/step - loss: 1016978678326432064602112.0000 - val_loss: 1134468995925614390149120.0000
Epoch 5/1000
128/128 [==============================] - 3s 20ms/step - loss: 1016978681163473785716736.0000 - val_loss: 1134468995925614390149120.0000
Epoch 6/1000
128/128 [==============================] - 3s 20ms/step - loss: 1016978682293527865458688.0000 - val_loss: 1134468995925614390149120.0000
Epoch 7/1000
128/128 [==============================] - 3s 21ms/step - loss: 1016978683996855255171072.0000 - val_

In [13]:
DNN = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(256, activation='relu'),
      layers.Dense(labels.shape[1], activation='softmax'),
    ])

In [14]:
train_encoded = np.array(autoencoder.encoder(train_x))
val_encoded = np.array(autoencoder.encoder(val_x))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [15]:
DNN.compile(optimizer='adam', loss=losses.CategoricalCrossentropy())

In [16]:
DNN.fit(train_encoded, train_y,
                epochs=1000,
                shuffle=True,
                validation_data=(val_encoded, val_y))

Train on 13999 samples, validate on 5938 samples
Epoch 1/1000
13999/13999 [==============================] - 4s 257us/sample - loss: 0.6038 - val_loss: 0.4183
Epoch 2/1000
13999/13999 [==============================] - 3s 212us/sample - loss: 0.2611 - val_loss: 0.2185
Epoch 3/1000
13999/13999 [==============================] - 3s 217us/sample - loss: 0.1956 - val_loss: 0.1939
Epoch 4/1000
13999/13999 [==============================] - 3s 216us/sample - loss: 0.1603 - val_loss: 0.1579
Epoch 5/1000
13999/13999 [==============================] - 3s 209us/sample - loss: 0.1377 - val_loss: 0.1582
Epoch 6/1000
13999/13999 [==============================] - 3s 208us/sample - loss: 0.1223 - val_loss: 0.1465
Epoch 7/1000
13999/13999 [==============================] - 3s 212us/sample - loss: 0.1055 - val_loss: 0.1203
Epoch 8/1000
13999/13999 [==============================] - 3s 207us/sample - loss: 0.0995 - val_loss: 0.1310
Epoch 9/1000
13999/13999 [==============================] - 3s 212us/sa

In [17]:
test_encoder = np.array(autoencoder.encoder(test_features))
y_ = DNN(test_encoder)

In [18]:
np.argmax(np.array(y_[2]))

12